In [207]:
import os

import pandas as pd
import numpy as np

In [208]:
path = r"E:\_Projects\gop\gop_bundle\powerquery"

In [209]:
# append curr GOP into gop2022
# a = pd.read_excel(os.path.join(path, "gop2022.xlsx"))
# b = pd.read_excel(os.path.join(path, "curr.xlsx"))
# pd.concat([a, b]).to_excel(os.path.join(path, "gop2022.xlsx"), index=False)
# del a, b

In [210]:
gops = ["21Plan.xlsx", "22Plan.xlsx", "gop2022.xlsx", "curr.xlsx", "prior.xlsx"]

In [211]:
dfs = []
for gop in gops:
    dfs.append(pd.read_excel(os.path.join(path, gop), engine='openpyxl').iloc[:,:24])

In [212]:
print(dfs)

[     Site Scenario     Sheetname Product Group                 Offer PF  \
0     BUD   21Plan      Budapest           ATM        Self Serv 20 - CD   
1     BUD   21Plan      Budapest           ATM        Self Serv 20 - CD   
2     BUD   21Plan      Budapest           ATM        Self Serv 20 - CD   
3     BUD   21Plan      Budapest           ATM        Self Serv 20 - CD   
4     BUD   21Plan      Budapest           ATM        Self Serv 20 - CD   
...   ...      ...           ...           ...                      ...   
1903  USL   21Plan  2021 EMS MPP         ITEMS  NON-Manufacturing Items   
1904  USL   21Plan  2021 EMS MPP         ITEMS  NON-Manufacturing Items   
1905  USL   21Plan  2021 EMS MPP         ITEMS  NON-Manufacturing Items   
1906  USL   21Plan  2021 EMS MPP         ITEMS  NON-Manufacturing Items   
1907  USL   21Plan  2021 EMS MPP         ITEMS  NON-Manufacturing Items   

                Range Class GOP Class  SSD       Date  ...  Exclude  \
0     NCR SelfServ CD  2012

In [213]:
dfs[4].columns

Index(['Site', 'Scenario', 'Sheetname', 'Product Group', 'Offer PF', 'Range',
       'Class', 'GOP Class', 'SSD', 'Date', 'Accounting Period', 'SSD_Qtr',
       'Year', 'Qty', 'Exclude', 'ExcludeWSP', 'Prod Range',
       'Offer Prod Portfolio', 'Grp', 'Flag_Discon', 'Flag_Exclude',
       'Flag_USI ', 'Show/Hide', 'MDAYS'],
      dtype='object')

In [214]:
lst_period = ["2019Q1","2019Q2","2019Q3","2019Q4","2020Q1","2020Q2","2020Q3","2020Q4","2021Q1","2021Q2","2021Q3","2021Q4","2022Q1","2022Q2","2022Q3","2022Q4","2023Q1","2023Q2","2023Q3","2023Q4","2024Q1","2024Q2","2024Q3","2024Q4","2025Q1","2025Q2","2025Q3","2025Q4","2026Q1","2026Q2","2026Q3","2026Q4"]
lst_year = ["2019","2020","2021","2022","2023","2024","2025","2026"]

for df in dfs:
    df["Qty"] = df["Qty"].replace('0x17',0.0)
    for ssd_qtr in lst_period:
        df[ssd_qtr] = np.where(df["SSD_Qtr"] == ssd_qtr, df["Qty"], 0)
    for ssd_year in lst_year:
        df[ssd_year] = np.where(df["Year"] == ssd_year, df["Qty"], 0)
        
# for df in dfs:
#     df["Qty"] = df["Qty"].replace('0x17',0.0)
#     for ssd_qtr in df["SSD_Qtr"].drop_duplicates():
#         df[ssd_qtr] = np.where(df["SSD_Qtr"] == ssd_qtr, df["Qty"], 0)
#     for ssd_year in df["Year"].drop_duplicates():
#         df[ssd_year] = np.where(df["Year"] == ssd_year, df["Qty"], 0)

In [215]:
gop_bundle = pd.concat(dfs[:4])

In [216]:
cols = list(dfs[2].columns)
new_cols = filter(lambda x: x not in ("Scenario"), cols[:cols.index("MDAYS")])

In [217]:
dims = list(new_cols)
facts = cols[cols.index("MDAYS")+1:]

In [218]:
print(dims)
print(facts)

['Site', 'Sheetname', 'Product Group', 'Offer PF', 'Range', 'Class', 'GOP Class', 'SSD', 'Date', 'Accounting Period', 'SSD_Qtr', 'Year', 'Qty', 'Exclude', 'ExcludeWSP', 'Prod Range', 'Offer Prod Portfolio', 'Grp', 'Flag_Discon', 'Flag_Exclude', 'Flag_USI ', 'Show/Hide']
['2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1', '2025Q2', '2025Q3', '2025Q4', '2026Q1', '2026Q2', '2026Q3', '2026Q4', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026']


In [219]:
facts_not_2019 = [fact for fact in facts if ('2019') not in str(fact)]
facts_not_2019_2020 = [fact for fact in facts_not_2019 if ('2020') not in str(fact)]

prior_grp = dfs[4][dims + facts_not_2019_2020].fillna(0).groupby(dims).sum().reset_index()
curr_grp = dfs[3][dims + facts_not_2019_2020].fillna(0).groupby(dims).sum().reset_index()


# curr_grp = dfs[3][dims].fillna(0).groupby(dims).sum().reset_index()
# prior_grp = dfs[4][dims].fillna(0).groupby(dims).sum().reset_index()

In [220]:
# curr_grp['Date'] = curr_grp['Date'].astype('datetime64[ns]')
# prior_grp['Date'] = prior_grp['Date'].astype('datetime64[ns]')
curr_grp.to_excel(os.path.join(path, "chngs_Trailcurr.xlsx"), index=False)
prior_grp.to_excel(os.path.join(path, "chngs_Trailprior.xlsx"), index=False)

In [221]:
# facts_not_2019 = [fact for fact in facts if '2019' not in str(fact)]

# curr_grp = dfs[3][dims + facts_not_2019].fillna(0).groupby(dims).sum().reset_index()
# prior_grp = dfs[4][dims + facts_not_2019].fillna(0).groupby(dims).sum().reset_index()

In [222]:
chgs_grp = curr_grp.merge(prior_grp, how="outer", on=dims, validate="1:1")
# print(chgs_grp)
# chgs_grp.to_excel(os.path.join(path, "chngs_Trailhari.xlsx"), index=False)
curr_grp.to_excel(os.path.join(path, "chngs_Trailcurr2.xlsx"), index=False)
prior_grp.to_excel(os.path.join(path, "chngs_Trailprior2.xlsx"), index=False)


In [223]:
for ssd_qtr in chgs_grp["SSD_Qtr"].drop_duplicates():
    chgs_grp[ssd_qtr] = chgs_grp[ssd_qtr+"_x"].fillna(0) - chgs_grp[ssd_qtr+"_y"].fillna(0)
for ssd_year in chgs_grp["Year"].drop_duplicates():
    chgs_grp[ssd_year] = chgs_grp[str(ssd_year)+"_x"].fillna(0) - chgs_grp[str(ssd_year)+"_y"].fillna(0)

In [224]:
cols = list(chgs_grp.columns)
new_cols = filter(lambda x: str(x)[-2:] not in ("_x", "_y"), cols)

In [225]:
chgs_grp = chgs_grp[list(new_cols)]
chgs_grp["Scenario"] = "Changes"

In [226]:
chgs_grp.to_excel(os.path.join(path, "chngs_tmp.xlsx"), index=False)

In [227]:
pd.concat([gop_bundle, chgs_grp]).to_excel(os.path.join(path, "gop_bundle.xlsx"), index=False)